Display search notebook component?



In [ ]:
import {Component, ModuleWithProviders, NgModule} from '@angular/core';
import {COMMON_MODULES} from '../../../imports/core.module';
import {RouterModule, Routes} from '@angular/router';
import {SearchService} from '../../../imports/search.service';

@Component({
    selector: 'bc-search',
    template: `
        <form>
            <md-input-container>
                <input mdInput name="search" required type="text"
                       placeholder="Search"
                       maxlength="100" [(ngModel)]="query" (change)="search()">
            </md-input-container>
        </form>
    `,
    styles: [`
        md-input-container {
            width: 100%;
        }
    `]
})
export class SearchComponent {
    query = '';
    service: SearchService

    constructor(public) {}

    search(): void {
        this.service.search(this.query).subscribe(r => {
            console.log('searched ' + r);
        });
    }
}

export const COMPONENTS = [
    SearchComponent
];

export const authRoutes: Routes = [
    {
        path: '',
        component: SearchComponent,
        data: {roles: ['anonymous', 'user']}
    }
];
export const routing: ModuleWithProviders = RouterModule.forChild(authRoutes);

@NgModule({
    imports: [
        ...COMMON_MODULES,
        routing
    ],
    declarations: COMPONENTS,
    exports: COMPONENTS
})
export class SearchModule {
}



Search notebook service?



In [ ]:
import {Observable} from 'rxjs/Observable';
import {Http, Response} from '@angular/http';
import {Injectable} from '@angular/core';

export let callbackUrl = 'localhost';

@Injectable()
export class SearchService {
    http: Http

    constructor(public
) {
}

search(query
:
string
):
Observable < Response > {
    console.log('Searching ' + query);
return this.http.post(callbackUrl, {query});
}
}


Display code results?

In [ ]:
import {SearchService} from '../../../imports/search.service';
import {ChangeDetectorRef, Component, OnDestroy, OnInit} from '@angular/core';
import {Subscription} from 'rxjs/Subscription';
import * as Prism from 'prismjs';

@Component({
    selector: 'bc-results',
    template: `
        <pre [class]="'language-'+(r.lang||'javascript')" *ngFor="let r of results" [innerHTML]="highlight(r.code)">
        </pre>
    `,
    styleUrls: ['./results.component.scss']
})
export class ResultsComponent implements OnInit, OnDestroy {
    query = '';
    results: Array<any> = [];
    private resultsSub: Subscription;
    

    constructor(public service: SearchService, public ref: ChangeDetectorRef) {}

    ngOnInit(): void {
        this.resultsSub = this.service.results(this.query).subscribe(r => {
            this.results = [(r as Array < any >)[0]];
            this.ref.detectChanges();
        });
    }

    ngOnDestroy(): void {
        this.resultsSub.unsubscribe();
    }

    highlight(code: string): string {
        return Prism.highlight(code, Prism.languages.javascript);
    }
}



Sockify search notebook provider?

In [ ]:
var client = require('socket.io-client')('http://localhost:8000', {
    secure: true,
    rejectUnauthorized: false
});
var importer = require('../Core');
var {
    fuseSearch,
    interpretObject,
} = importer.import('interpret all notebooks.ipynb')
var searchNotebooks = importer.import('search notebooks using gulp');

var fuseSearch, interpretObject;
function searchAll(queries, root = '') {
    return (typeof queries === 'string'
            ? fuseSearch(queries)
            : Promise.all(queries.map(fuseSearch)))
        .then(r => interpretObject(r))
};

function searchHandler() {
    return new Promise(resolve => {
        console.log('Search handler');
        client.on('resolve', (name, search) => {
            console.log('searching...');
            if (name === 'SearchService.prototype.search') {
                client.emit('result', 'SearchService.prototype.search',
                    'Searching notebooks for ' + search);
                // TODO: include gulp notebook search, .bash_sessions,
                //    selenium scripts, stack overflow, github, 
                Promise.all([
                    searchAll(search),
                    searchNotebooks(search)
                ])
                    .then(r => client.emit('result', 'SearchService.prototype.results', [].concat(...r)))
                    .catch(e => console.log(e))
            }
        });
        client.emit('handle', 'SearchService', () => resolve());
        client.on('error', e => console.log(e))
    })
};
module.exports = searchHandler;



[Function: searchHandler]

In [1]:
var PROFILE_PATH = process.env.HOME || process.env.HOMEPATH || process.env.USERPROFILE;
var project = PROFILE_PATH + '/universal';
var server = require(project + '/src/server/sockify-server.js').sockifyServer(8000);
var client = require('socket.io-client').connect('http://localhost:8000');
client.emit('close');


Socket {
  io: 
   Manager {
     nsps: { '/': [Circular] },
     subs: [ [Object], [Object], [Object] ],
     opts: 
      { path: '/socket.io',
        hostname: 'localhost',
        secure: true,
        port: '8000' },
     _reconnection: true,
     _reconnectionAttempts: Infinity,
     _reconnectionDelay: 1000,
     _reconnectionDelayMax: 5000,
     _randomizationFactor: 0.5,
     backoff: Backoff { ms: 1000, max: 5000, factor: 2, jitter: 0.5, attempts: 0 },
     _timeout: 20000,
     readyState: 'opening',
     uri: 'http://localhost:8000',
     connecting: [ [Circular] ],
     lastPing: null,
     encoding: false,
     packetBuffer: [],
     encoder: Encoder {},
     decoder: Decoder { reconstructor: null },
     autoConnect: true,
     engine: 
      Socket {
        secure: true,
        agent: false,
        hostname: 'localhost',
        port: '8000',
        query: {},
        upgrade: true,
        path: '/socket.io/',
        forceJSONP: false,
        jsonp: true,
      

Error: listen EADDRINUSE :::8000
    at Object.exports._errnoException (util.js:1033:11)
    at exports._exceptionWithHostPort (util.js:1056:20)
    at Server._listen2 (net.js:1276:14)
    at listen (net.js:1312:10)
    at Server.listen (net.js:1403:7)
    at Server.listen.Server.attach (C:\Users\brian.cullinan\universal\node_modules\socket.io\lib\index.js:237:9)
    at new Server (C:\Users\brian.cullinan\universal\node_modules\socket.io\lib\index.js:56:17)
    at Function.Server [as listen] (C:\Users\brian.cullinan\universal\node_modules\socket.io\lib\index.js:42:41)
    at Object.sockifyServer (C:\Users\brian.cullinan\universal\src\server\sockify-server.js:43:17)
    at evalmachine.<anonymous>:5:65

Display login form?



In [ ]:
import {RouterModule, Routes} from '@angular/router';
import {Component, ModuleWithProviders, NgModule} from '@angular/core';
import {COMMON_MODULES} from '../../../imports/core.module';
import {AuthService} from '../../../imports/auth.service';

@Component({
    selector: 'bc-login',
    template: `
        <form>
            <md-input-container>
                <input mdInput name="signupEmail" required type="email"
                       placeholder="Username"
                       maxlength="100" [(ngModel)]="username">
            </md-input-container>
            <br/>
            <md-input-container>
                <input mdInput name="signupPassword" required type="password" maxlength="25"
                       placeholder="Password"
                       length="40" [(ngModel)]="password">
            </md-input-container>
            <button md-raised-button color="primary"
                    id="signupSubmit" type="submit" (click)="onLogin()">
                Login
            </button>
        </form>
    `,
    styleUrls: ['./login.component.scss']
})
export class LoginComponent {
    username: string;
    password: string;
    service: AuthService

    constructor(public
) {
}

onLogin()
:
void {
    this.service.login(this.username, this.password).subscribe(r => {
    console.log(r);
});
}
}

export const COMPONENTS = [
    LoginComponent
];

export const authRoutes: Routes = [
    {
        path: '',
        component: LoginComponent,
        data: {roles: ['anonymous', 'user']}
    }
];
export const routing: ModuleWithProviders = RouterModule.forChild(authRoutes);

@NgModule({
    imports: [
        ...COMMON_MODULES,
        routing
    ],
    declarations: COMPONENTS,
    exports: COMPONENTS
})
export class AuthModule {
}



Authentication service?
    


In [ ]:
import {Observable} from 'rxjs/Observable';
import {Http} from '@angular/http';
import {Injectable} from '@angular/core';

export let callbackUrl = 'localhost';

@Injectable()
export class AuthService {
    http: Http

    constructor(public
) {
}

login(username
:
string, password
:
string
):
Observable < number > {
    return this.http.post(callbackUrl, {username, password})
        .map(r => r.json());
}
}

